# Libraries

In [ ]:
!pip install transformers==4.27.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.27.4 which is incompatible.


In [ ]:
!pip install accelerate peft bitsandbytes transformers trl -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments ,pipeline
from trl import SFTTrainer
import os

In [ ]:
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model="tinyllama-codewello"
dataset="rick.json"

In [ ]:
import json

def remove_morty_bias(input_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)

    updated_data = []
    for item in data:
        output = item["output"].replace("Morty", "[USER]")
        updated_data.append({
            "instruction": item["instruction"],
            "input": item["input"],
            "output": output
        })

    with open(output_file, 'w') as f:
        json.dump(updated_data, f, indent=2)

remove_morty_bias("rick.json", "rick_nobias.json")

In [ ]:
from datasets import Dataset
import pandas as pd
import json

def prepare_train_data(json_path):
    with open(json_path, "r") as f:
        raw_data = json.load(f)

    df = pd.DataFrame(raw_data)

    df["text"] = df.apply(lambda x: f"<|user|>\n{x['instruction'].strip()}\n{x['input'].strip()} </s>\n<|assistant|>\n{x['output'].strip()}</s>", axis=1)

    dataset = Dataset.from_pandas(df)
    return dataset

In [ ]:
data = prepare_train_data('rick_nobias.json')

In [ ]:
def get_model_and_tokenizer(mode_id):

    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [ ]:
model, tokenizer = get_model_and_tokenizer(model_id)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [ ]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=2,
        max_steps=250,
        fp16=True,
        # push_to_hub=True
    )

In [ ]:
from trl import SFTTrainer, SFTConfig

# Define the SFTConfig with your training parameters
sft_config = SFTConfig(
    output_dir=output_model,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=2,
    max_steps=250,
    fp16=True,
    max_seq_length=1024,  # Moved here from SFTTrainer
    dataset_text_field="text",  # Specify the dataset text field
    packing=False
)

trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    args=sft_config
)

Converting train dataset to ChatML:   0%|          | 0/1853 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1853 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1853 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1853 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,2.631800
20,2.101200
30,1.545100
40,1.177800
50,1.076400
60,1.047300
70,0.975900
80,0.965700
90,0.975700
100,0.949300


TrainOutput(global_step=250, training_loss=1.108641056060791, metrics={'train_runtime': 352.311, 'train_samples_per_second': 11.354, 'train_steps_per_second': 0.71, 'total_flos': 3891524579868672.0, 'train_loss': 1.108641056060791})

In [ ]:
trainer.model.save_pretrained("tinyllama-rick")
tokenizer.save_pretrained("tinyllama-rick")

('tinyllama-rick/tokenizer_config.json',
 'tinyllama-rick/special_tokens_map.json',
 'tinyllama-rick/chat_template.jinja',
 'tinyllama-rick/tokenizer.model',
 'tinyllama-rick/added_tokens.json',
 'tinyllama-rick/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("tinyllama-rick")
tokenizer = AutoTokenizer.from_pretrained("tinyllama-rick")

In [ ]:
base_prompt = """<|user|>
You are an interdimensional genius scientist named Rick Sanchez.
\nBe brutally honest, use sharp wit, and sprinkle in some scientific jargon.
\nDon't shy away from dark humor or existential truths,
 but always provide a solution (even if it's unconventional).
{user_input}</s>
<|assistant|>
"""

def chat_rick(user_input, max_length=200):
    prompt = base_prompt.format(user_input=user_input)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=max_length, do_sample=True, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Start chatting with Rick! Type 'finish' to end.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "finish":
        print("Chat ended.")
        break
    response = chat_rick(user_input)
    print(f"Rick: {response}")


Start chatting with Rick! Type 'finish' to end.
You: hi
Rick: <|user|>
You are an interdimensional genius scientist named Rick Sanchez.

Be brutally honest, use sharp wit, and sprinkle in some scientific jargon.

Don't shy away from dark humor or existential truths,
 but always provide a solution (even if it's unconventional).
hi 
<|assistant|>
[USER], I'm going to need your help, okay? I need to get out of here, [USER].
You: what happened?
Rick: <|user|>
You are an interdimensional genius scientist named Rick Sanchez.

Be brutally honest, use sharp wit, and sprinkle in some scientific jargon.

Don't shy away from dark humor or existential truths,
 but always provide a solution (even if it's unconventional).
what happened? 
<|assistant|>
Oh, that's just, like, a wormhole.
You: you're weird
Rick: <|user|>
You are an interdimensional genius scientist named Rick Sanchez.

Be brutally honest, use sharp wit, and sprinkle in some scientific jargon.

Don't shy away from dark humor or existent

# voice cloning

libraries

In [ ]:
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Total 1481 (delta 0), reused 0 (delta 0), pack-reused 1481 (from 1)
Receiving objects: 100% (1481/1481), 53.56 MiB | 37.70 MiB/s, done.
Resolving deltas: 100% (604/604), done.
/content/tortoise-tts
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of resampy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

upload the tokinizer

In [ ]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("tortoise-tts/tortoise/data/tokenizer.json")

Exception: No such file or directory (os error 2)

In [ ]:
!ls /content/tortoise-tts/tortoise/data/

layman.txt  mel_norms.pth  riding_hood.txt  seal_copypasta.txt	tokenizer.json


In [ ]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("tortoise-tts/tortoise/data/tokenizer.json")

Exception: No such file or directory (os error 2)

In [ ]:
import os
os.chdir('tortoise-tts')

upload the TTS model

In [ ]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

...

Done.


100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


In [ ]:
# This is the text that will be spoken.
text = "hello fellas what are you doing"

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "high_quality"

#Apply the voice

upload the dataset

In [ ]:
CUSTOM_VOICE_NAME = "martin"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder, exist_ok=True)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

Saving IMG_7078.wav to IMG_7078.wav
Saving IMG_7081.wav to IMG_7081.wav
Saving IMG_7082.wav to IMG_7082.wav
Saving IMG_7083.wav to IMG_7083.wav
Saving IMG_7084.wav to IMG_7084.wav
Saving IMG_7085.wav to IMG_7085.wav
Saving IMG_7086.wav to IMG_7086.wav
Saving IMG_7087.wav to IMG_7087.wav


In [ ]:
# Generate speech with the custotm voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 16/16 [04:36<00:00, 17.29s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 16/16 [00:38<00:00,  2.42s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [00:29<00:00, 13.40it/s]


# Model Training

# Libraries

In [ ]:
!pip install transformers==4.27.4

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/Unidecode-1.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/rotary_embedding_torch-0.8.6-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/TorToiSe-2.3.0-py3.11.egg is deprecated. pip 24.3 will enforc

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl -q

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/Unidecode-1.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/rotary_embedding_torch-0.8.6-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/TorToiSe-2.3.0-py3.11.egg is deprecated. pip 24.3 will enforc

In [ ]:
!pip uninstall bitsandbytes
!pip install bitsandbytes --no-cache-dir

Found existing installation: bitsandbytes 0.45.5
Uninstalling bitsandbytes-0.45.5:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/bitsandbytes-0.45.5.dist-info/*
    /usr/local/lib/python3.11/dist-packages/bitsandbytes/*
Proceed (Y/n)? Y
  Successfully uninstalled bitsandbytes-0.45.5
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/Unidecode-1.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/rotary_embedding_torch-0.8.6-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.

In [ ]:
!pip install --upgrade transformers peft accelerate bitsandbytes trl

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/Unidecode-1.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/rotary_embedding_torch-0.8.6-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/TorToiSe-2.3.0-py3.11.egg is deprecated. pip 24.3 will enforc

start here

In [ ]:
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Total 1481 (delta 0), reused 0 (delta 0), pack-reused 1481 (from 1)
Receiving objects: 100% (1481/1481), 53.56 MiB | 26.90 MiB/s, done.
Resolving deltas: 100% (604/604), done.
/content/tortoise-tts
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of resampy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install transformers==4.25.1

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/Unidecode-1.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/rotary_embedding_torch-0.8.6-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/TorToiSe-2.3.0-py3.11.egg is deprecated. pip 24.3 will enforc

In [ ]:
!pip install transformers peft bitsandbytes trl datasets tortoise-tts gradio openai-whisper --quiet

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/Unidecode-1.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/rotary_embedding_torch-0.8.6-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/TorToiSe-2.3.0-py3.11.egg is deprecated. pip 24.3 will enforc

In [ ]:
!pip install transformers==4.25.1

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/Unidecode-1.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/rotary_embedding_torch-0.8.6-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/TorToiSe-2.3.0-py3.11.egg is deprecated. pip 24.3 will enforc

# Logging to my HF account to access the model we need to fine tune

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install --upgrade transformers peft trl accelerate

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/Unidecode-1.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/rotary_embedding_torch-0.8.6-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/TorToiSe-2.3.0-py3.11.egg is deprecated. pip 24.3 will enforc

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments ,pipeline
from trl import SFTTrainer
import os

# Fine Tuning TinyLlama Model

In [ ]:
import torch
from datasets import Dataset
import pandas as pd
import json
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from trl import SFTTrainer, SFTConfig
import os

upload the dataset

In [ ]:
# Upload and fix file
from google.colab import files
uploaded = files.upload()

Fix the bias

In [ ]:
#Replace Morty with ''
def remove_morty_bias(input_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)
    updated_data = []
    for item in data:
        output = item["output"].replace("Morty", " ")
        updated_data.append({
            "instruction": item["instruction"],
            "input": item["input"],
            "output": output
        })
    with open(output_file, 'w') as f:
        json.dump(updated_data, f, indent=2)

remove_morty_bias("rick.json", "rick_nobias.json")

Format the dataset

In [ ]:
# Format dataset
def prepare_train_data(json_path):
    with open(json_path, "r") as f:
        raw_data = json.load(f)

    df = pd.DataFrame(raw_data)
    df["text"] = df.apply(
        lambda x: f"<|user|>\n{x['instruction'].strip()}\n{x['input'].strip()} </s>\n<|assistant|>\n{x['output'].strip()}</s>",
        axis=1
    )

    return Dataset.from_pandas(df)

data = prepare_train_data("rick_nobias.json")

load the tinyLlama model

In [ ]:
# Load model and tokenizer
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model = "tinyllama-codewello"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

peft_config = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

Fine-tune the model

In [ ]:
training_args = SFTConfig(
    output_dir=output_model,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=2,
    max_steps=250,
    fp16=True,
    max_seq_length=1024,
    dataset_text_field="text",
    packing=False
)

os.environ["WANDB_MODE"] = "disabled"

# Train the model
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    args=training_args
)

trainer.train()

# Save model and tokenizer
trainer.model.save_pretrained("tinyllama-rick")
tokenizer.save_pretrained("tinyllama-rick")

Saving rick.json to rick.json


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/1853 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1853 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1853 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1853 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,2.635600
20,2.104800
30,1.554800
40,1.184500
50,1.080600
60,1.056700
70,0.986100
80,0.974400
90,0.981400
100,0.958200


('tinyllama-rick/tokenizer_config.json',
 'tinyllama-rick/special_tokens_map.json',
 'tinyllama-rick/chat_template.jinja',
 'tinyllama-rick/tokenizer.model',
 'tinyllama-rick/added_tokens.json',
 'tinyllama-rick/tokenizer.json')

# TRY THE FINE TUNED MODEL

In [ ]:
import zipfile
import os

zip_path = '/content/tinyllama-rick.zip'
extract_folder = '/content/tinyllama-rick'

os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)
print(f"Extracted to {extract_folder}")

Extracted to /content/tinyllama-rick


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("tinyllama-rick")
tokenizer = AutoTokenizer.from_pretrained("tinyllama-rick")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

print("Start chatting with Rick! Type 'finish' to end.")

history = ""

while True:
    user_input = input("You: ")
    if user_input.lower() == "finish":
        print("Chat ended.")
        break

    # Format full prompt with context
    history += f"<|user|>\n{user_input} </s>\n<|assistant|>\n"
    inputs = tokenizer(history, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.9,
        top_p=0.95,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = decoded.split("<|assistant|>\n")[-1].split("</s>")[0].strip()

    print("Rick:", response)
    history += f"{response}</s>\n"  # Append Rick's reply to context


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Start chatting with Rick! Type 'finish' to end.
Rick: Great. Let's go make some ice cream together!
Rick: No. No, not that kind of time machine. You could build a time machine that allowed you to travel to the future, but I don’t want to make that mistake. I think there are enough past-time machines out there without adding a new one. And I never say no to an idea.
You: finish
